Goals: 
- Deal with null data points   
- Clean the data (getting rid of the units)
- Data split (train / test)
- Save the clean data in data/processed 

In [505]:
import pandas as pd

In [506]:
file_loc = "../data/raw/Car details v3.csv"
df = pd.read_csv(file_loc)
df.notna().sum()

name             8128
year             8128
selling_price    8128
km_driven        8128
fuel             8128
seller_type      8128
transmission     8128
owner            8128
mileage          7907
engine           7907
max_power        7913
torque           7906
seats            7907
dtype: int64

In [507]:
8128-7906 #=222
(8128-7906)/8128*100

2.731299212598425

Torque has the least number of Non-none values. If we were to do dropNA, we would lose at least $8128-7906 = 222$ values. This will correspond to $222/8128*100 = 2.73\%$ data loss.  

In [508]:
df = df.dropna()
#df.notna().sum()
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7906 entries, 0 to 8127
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           7906 non-null   object 
 1   year           7906 non-null   int64  
 2   selling_price  7906 non-null   int64  
 3   km_driven      7906 non-null   int64  
 4   fuel           7906 non-null   object 
 5   seller_type    7906 non-null   object 
 6   transmission   7906 non-null   object 
 7   owner          7906 non-null   object 
 8   mileage        7906 non-null   object 
 9   engine         7906 non-null   object 
 10  max_power      7906 non-null   object 
 11  torque         7906 non-null   object 
 12  seats          7906 non-null   float64
dtypes: float64(1), int64(3), object(9)
memory usage: 864.7+ KB


In the dataframe, NA values are overlapping. That is, many rows have NA column entries. We will drop those rows.

In [509]:
df.sample(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
5231,Toyota Corolla AE,2004,135000,90000,Petrol,Individual,Manual,Second Owner,13.4 kmpl,1794 CC,125 bhp,"16.1@ 4,200(kgm@ rpm)",5.0
2428,Ford Fiesta Diesel Style,2012,280000,60000,Diesel,Individual,Manual,Second Owner,23.5 kmpl,1498 CC,88.7 bhp,204Nm@ 2000-2750rpm,5.0
7235,Maruti Wagon R AMT VXI,2017,430000,20000,Petrol,Individual,Automatic,First Owner,20.51 kmpl,998 CC,67 bhp,90Nm@ 3500rpm,5.0
700,Renault KWID 1.0 RXT Optional,2016,300000,50000,Petrol,Individual,Manual,First Owner,23.01 kmpl,999 CC,67 bhp,91Nm@ 4250rpm,5.0
50,Honda Amaze S Diesel,2015,425000,49800,Diesel,Dealer,Manual,First Owner,24.7 kmpl,1498 CC,98.63 bhp,200Nm@ 1750rpm,5.0


The columns "selling_price", "km_driven", "year", and "seats" have float values. Therefore, no need to change those. The columns "mileage", "engine", and "max_power" have units. We may need to do unit conversion and coversion to float values. The "torque" column is weird, need to pay special attention. The columns "fuel", "seller_type", "transmission", and "owner" are categorical features.

First, we will work on the "engine" column

In [510]:
df["engine"].apply(lambda string : string.split(" ")[1]).unique()

array(['CC'], dtype=object)

The "engine" column has only "CC" (cubic centimeters) unit. We only need to grab the corresponding value.

In [511]:
df["engine_value_cc"] = df["engine"].apply(lambda string : float(string.split(" ")[0]))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7906 entries, 0 to 8127
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             7906 non-null   object 
 1   year             7906 non-null   int64  
 2   selling_price    7906 non-null   int64  
 3   km_driven        7906 non-null   int64  
 4   fuel             7906 non-null   object 
 5   seller_type      7906 non-null   object 
 6   transmission     7906 non-null   object 
 7   owner            7906 non-null   object 
 8   mileage          7906 non-null   object 
 9   engine           7906 non-null   object 
 10  max_power        7906 non-null   object 
 11  torque           7906 non-null   object 
 12  seats            7906 non-null   float64
 13  engine_value_cc  7906 non-null   float64
dtypes: float64(2), int64(3), object(9)
memory usage: 926.5+ KB


Do the same for max power.

In [512]:
df["max_power"].apply(lambda string: string.split(" ")[1]).unique()

array(['bhp'], dtype=object)

In [513]:
df["max_power_bph"] = df["max_power"].apply(lambda string: float(string.split(" ")[0]))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7906 entries, 0 to 8127
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             7906 non-null   object 
 1   year             7906 non-null   int64  
 2   selling_price    7906 non-null   int64  
 3   km_driven        7906 non-null   int64  
 4   fuel             7906 non-null   object 
 5   seller_type      7906 non-null   object 
 6   transmission     7906 non-null   object 
 7   owner            7906 non-null   object 
 8   mileage          7906 non-null   object 
 9   engine           7906 non-null   object 
 10  max_power        7906 non-null   object 
 11  torque           7906 non-null   object 
 12  seats            7906 non-null   float64
 13  engine_value_cc  7906 non-null   float64
 14  max_power_bph    7906 non-null   float64
dtypes: float64(3), int64(3), object(9)
memory usage: 988.2+ KB


Now, working on the "mileage" column.

In [514]:
df["mileage"].apply(lambda string: string.split(" ")[1]).unique()

array(['kmpl', 'km/kg'], dtype=object)

We want to convert to a common unit. We will convert km/kg to kmpl. We need to check which fuel uses kmpl and which used km/kg. Then, we will multiply by density of the fluid. $$\frac{km}{L} = \frac{km}{kg} * \frac{kg}{L}. $$

In [515]:
fuel_types = df["fuel"].unique() #['Diesel' 'Petrol' 'LPG' 'CNG']

for entry in fuel_types:
    fuel_units = df[df["fuel"] == entry]["mileage"].apply(lambda string : string.split(" ")[1]).unique()
    print(f"{entry:<20}{fuel_units}")

Diesel              ['kmpl']
Petrol              ['kmpl']
LPG                 ['km/kg']
CNG                 ['km/kg']


We need to find the density of LPG and CNG.

In [516]:
density_LPG = 0.4*1.96 + 0.6 * 1.724 # Kg/L from https://www.rippleinfra.com/lpg.html  Taking 4:6 ratio of propane : butane 

density_CNG = 200/1000 # check both values. 200 is kg/m^3

In [517]:
#making categorical variables for fuel types.

for entry in fuel_types:
    df[f"fuel_{entry}"] = df["fuel"].apply(lambda string : 1 if string == entry else 0)

df.sample(5)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,engine_value_cc,max_power_bph,fuel_Diesel,fuel_Petrol,fuel_LPG,fuel_CNG
7412,Maruti 800 AC,2012,100000,30000,Petrol,Individual,Manual,First Owner,16.1 kmpl,796 CC,37 bhp,59Nm@ 2500rpm,4.0,796.0,37.0,0,1,0,0
5152,Audi Q7 3.0 TDI Quattro,2009,1025000,120000,Diesel,Individual,Automatic,Third Owner,12.07 kmpl,2967 CC,241.4 bhp,550Nm@ 1750-2750rpm,7.0,2967.0,241.4,1,0,0,0
5756,Maruti Ertiga ZDI Plus,2019,1200000,14000,Diesel,Individual,Manual,First Owner,25.47 kmpl,1248 CC,88.50 bhp,200Nm@ 1750rpm,7.0,1248.0,88.5,1,0,0,0
915,Maruti Eeco Smiles 7 Seater Standard,2013,204999,100000,Petrol,Individual,Manual,Third Owner,15.1 kmpl,1196 CC,73 bhp,101Nm@ 3000rpm,7.0,1196.0,73.0,0,1,0,0
3946,Nissan Micra Diesel XL Optional,2013,400000,83000,Diesel,Individual,Manual,First Owner,23.08 kmpl,1461 CC,63.1 bhp,160Nm@ 2000rpm,5.0,1461.0,63.1,1,0,0,0


We will make a column called "mileage_conversion_factor". This column contain the conversion factor for each type of fuel. For example, for petrol we already have mileage in kmpl, therefore the conversion factor is 1. For CNG, it will be the density.

In [518]:
df["mileage_coversion_factor"] = df["fuel_Diesel"] + df["fuel_Petrol"] + density_CNG * df["fuel_CNG"] + density_LPG * df["fuel_LPG"] # multiply by this number to convert

Now convert all mileage values to kmpl.

In [519]:
df["mileage_kmpl"] = df["mileage"].apply(lambda string: float(string.split(" ")[0])) * df["mileage_coversion_factor"]
df.head(15)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,...,torque,seats,engine_value_cc,max_power_bph,fuel_Diesel,fuel_Petrol,fuel_LPG,fuel_CNG,mileage_coversion_factor,mileage_kmpl
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,...,190Nm@ 2000rpm,5.0,1248.0,74.00,1,0,0,0,1.0000,23.40000
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,...,250Nm@ 1500-2500rpm,5.0,1498.0,103.52,1,0,0,0,1.0000,21.14000
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,...,"12.7@ 2,700(kgm@ rpm)",5.0,1497.0,78.00,0,1,0,0,1.0000,17.70000
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,...,22.4 kgm at 1750-2750rpm,5.0,1396.0,90.00,1,0,0,0,1.0000,23.00000
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,...,"11.5@ 4,500(kgm@ rpm)",5.0,1298.0,88.20,0,1,0,0,1.0000,16.10000
5,Hyundai Xcent 1.2 VTVT E Plus,2017,440000,45000,Petrol,Individual,Manual,First Owner,20.14 kmpl,1197 CC,...,113.75nm@ 4000rpm,5.0,1197.0,81.86,0,1,0,0,1.0000,20.14000
6,Maruti Wagon R LXI DUO BSIII,2007,96000,175000,LPG,Individual,Manual,First Owner,17.3 km/kg,1061 CC,...,"7.8@ 4,500(kgm@ rpm)",5.0,1061.0,57.50,0,0,1,0,1.8184,31.45832
7,Maruti 800 DX BSII,2001,45000,5000,Petrol,Individual,Manual,Second Owner,16.1 kmpl,796 CC,...,59Nm@ 2500rpm,4.0,796.0,37.00,0,1,0,0,1.0000,16.10000
8,Toyota Etios VXD,2011,350000,90000,Diesel,Individual,Manual,First Owner,23.59 kmpl,1364 CC,...,170Nm@ 1800-2400rpm,5.0,1364.0,67.10,1,0,0,0,1.0000,23.59000
9,Ford Figo Diesel Celebration Edition,2013,200000,169000,Diesel,Individual,Manual,First Owner,20.0 kmpl,1399 CC,...,160Nm@ 2000rpm,5.0,1399.0,68.10,1,0,0,0,1.0000,20.00000


Next, we will extract data from the torque values. Unfortunately, the torque values have different types of units. From the first glance, there is Nm (Newton-meter) and kgm (kg-meter) for the torque. Also, there are rpm vales. We would like to know the kind of units we are given.

The following function takes a string and returns the same string without any numbers.

In [520]:
def eat_numbers(text:str ) -> str:
    """Takes a string and outputs the same string without numbers. Example: "No 1 DJ" -> "No  DJ", "50kg" -> "kg" """
    return ''.join(["" if char.isdigit() else char for char in text])

assert(eat_numbers("No 1 DJ") == "No  DJ")
assert(eat_numbers("115Nm@ 3500-3600rpm") == "Nm@ -rpm")

import re 

def replace_numbers_with_star(text : str) -> str:
    """Takes a string replaces a number sequence with *. Example: "No 10 DJ" -> "No * DJ", "50kg over 500m" -> "*kg over *m" """
    new_text =  re.sub(r'\d+', '*', text)
    return new_text.replace('*.*', '*').replace(' ', '').lower()

replace_numbers_with_star("50kg over 500m")

'*kgover*m'

Let's check what units are there.

In [521]:
df["torque"].apply(replace_numbers_with_star).unique()

array(['*nm@*rpm', '*nm@*-*rpm', '*@*,*(kgm@rpm)', '*kgmat*-*rpm',
       '*kgm@*rpm', '*nm@*~*rpm', '*nmat*rpm', '*@*-*rpm', '*nm',
       '*kgmat*rpm', '*kgmat*,*rpm', '*@*-*(kgm@rpm)', '*nmat*-*rpm',
       '*@*,*-*,*(kgm@rpm)', '*nm(*kgm)@*rpm', '*nm@*-*', '*nm@*+/-*rpm',
       '*@*,*+/-*(nm@rpm)', '*@*-*', '*(*)@*', '*nm/*rpm', '*@*(kgm@rpm)',
       '*nm@*,*rpm', '*nm@*', '*/*', '*nmat*,*-*,*rpm'], dtype=object)

In [522]:
df = df.dropna()
#df.info()
columns = list(df.columns)
categorical_features = ['fuel', 'seller_type', 'transmission', 'owner', 'seats']

for feature in categorical_features:
    print(f"{feature:<20} {df[feature].unique()}")

fuel                 ['Diesel' 'Petrol' 'LPG' 'CNG']
seller_type          ['Individual' 'Dealer' 'Trustmark Dealer']
transmission         ['Manual' 'Automatic']
owner                ['First Owner' 'Second Owner' 'Third Owner' 'Fourth & Above Owner'
 'Test Drive Car']
seats                [ 5.  4.  7.  8.  6.  9. 10. 14.  2.]


In [523]:
df[df["fuel"]=="CNG"].head(2)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,...,torque,seats,engine_value_cc,max_power_bph,fuel_Diesel,fuel_Petrol,fuel_LPG,fuel_CNG,mileage_coversion_factor,mileage_kmpl
35,Maruti Alto 800 CNG LXI Optional,2019,330000,10000,CNG,Individual,Manual,Second Owner,33.44 km/kg,796 CC,...,60Nm@ 3500rpm,4.0,796.0,40.30,0,0,0,1,0.2,6.688
203,Maruti Wagon R LXI CNG,2016,360000,50000,CNG,Individual,Manual,First Owner,26.6 km/kg,998 CC,...,77Nm@ 3500rpm,5.0,998.0,58.16,0,0,0,1,0.2,5.320


In [524]:
df["fuel"].unique()

array(['Diesel', 'Petrol', 'LPG', 'CNG'], dtype=object)

In [525]:
df["fuel_Diesel"] = pd.get_dummies(df["fuel"])["Diesel"]

In [526]:
for feature in categorical_features:
    for category in df[feature].unique():
        df[f"{feature}_{category}"] = pd.get_dummies(df[feature], dtype=int)[category]

df.head(15)

df.columns

Index(['name', 'year', 'selling_price', 'km_driven', 'fuel', 'seller_type',
       'transmission', 'owner', 'mileage', 'engine', 'max_power', 'torque',
       'seats', 'engine_value_cc', 'max_power_bph', 'fuel_Diesel',
       'fuel_Petrol', 'fuel_LPG', 'fuel_CNG', 'mileage_coversion_factor',
       'mileage_kmpl', 'seller_type_Individual', 'seller_type_Dealer',
       'seller_type_Trustmark Dealer', 'transmission_Manual',
       'transmission_Automatic', 'owner_First Owner', 'owner_Second Owner',
       'owner_Third Owner', 'owner_Fourth & Above Owner',
       'owner_Test Drive Car', 'seats_5.0', 'seats_4.0', 'seats_7.0',
       'seats_8.0', 'seats_6.0', 'seats_9.0', 'seats_10.0', 'seats_14.0',
       'seats_2.0'],
      dtype='object')

In [527]:
df["engine"].apply(lambda string : string.split(" ")[1]).unique()

array(['CC'], dtype=object)

In [528]:
df["engine_value_cc"] = df["engine"].apply(lambda string : float(string.split(" ")[0]))

In [529]:
df["max_power"].apply(lambda string: string.split(" ")[1]).unique()

array(['bhp'], dtype=object)

In [530]:
df["max_power_bph"] = df["max_power"].apply(lambda string: float(string.split(" ")[0]))

In [531]:
df["mileage"].apply(lambda string : string.split(" ")[1]).unique()

array(['kmpl', 'km/kg'], dtype=object)